In [135]:
from tables import *
import pandas as pd
import numpy as np
import tensorflow as tf

In [136]:
train = pd.read_hdf("train.h5", "train")
test = pd.read_hdf("test.h5", "test")

In [137]:
y_train = train['y']
x_train = train._drop_axis(['y'], axis=1)

#Switch to numpy
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(test)

x_train.shape

(45324, 100)

#### Convert to 1-Hot

In [138]:
#5 classes: 0,1,2,3,4
y_train_hot = np.zeros((y_train.shape[0], 5))
y_train_hot[np.arange(y_train.shape[0]), y_train] = 1

#see 1-hot
y_train_hot[0:5]

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.]])

#### Parameters

In [169]:
learning_rate = 0.001
epochs = 2000
batch_size = 16
disp_step = 100
logs_path = './tmp/tensorflow_logs' #for tensorboard

num_input = 100
n_hidden_1 = 70
n_hidden_2 = 40
n_hidden_3 = 60
n_hidden_4 = 25
num_classes = 5

#### Placeholders, fed by training data

In [170]:
with tf.name_scope('DATA'):
    X = tf.placeholder(tf.float32, [None, num_input], name='X')
    y = tf.placeholder(tf.float32, [None, num_classes], name='y')

#### TF Variables, to be trained

In [171]:
with tf.name_scope('VARIABLES'):
    weights = {
        'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
        'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4])),
        'out': tf.Variable(tf.random_normal([n_hidden_4, num_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'b3': tf.Variable(tf.random_normal([n_hidden_3])),
        'b4': tf.Variable(tf.random_normal([n_hidden_4])),
        'out': tf.Variable(tf.random_normal([num_classes]))
    }

In [172]:
def neural_net(x):    
    
    #with tf.name_scope('RELU'):
     #   layer_1 = tf.nn.relu_layer(X, weights['h1'], biases['b1'], name='relu1')
      #  layer_2 = tf.nn.relu_layer(layer_1, weights['h2'], biases['b2'], name='relu2')
       # out_layer = tf.nn.softmax(tf.matmul(layer_2, weights['out']) + biases['out'])
        
    #Just multilayer perceptron    
    layer_1 = tf.add(tf.matmul(X, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    
    #out_layer = tf.nn.softmax(tf.matmul(layer_3, weights['out']) + biases['out'])
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out'] 
    
    
    return out_layer

#### Define Loss and Optimizer

In [173]:
logits = neural_net(X)

with tf.name_scope('LOSS'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=y))
    tf.summary.scalar('loss', loss)
                          
with tf.name_scope('OPTIMIZER'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
                          
#Evaluation
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) 
                        
init = tf.global_variables_initializer() 

#x_train[(1*batch_size):(1+batch_size)+batch_size-1]

In [174]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        batch_xs = x_train[(epoch*batch_size):((epoch+1)*batch_size)-1]
        batch_ys = y_train_hot[(epoch*batch_size):((epoch+1)*batch_size)-1]
        
        
        #print(epoch, batch_xs.shape)
        sess.run(optimizer, feed_dict={X: batch_xs, y: batch_ys})
        
        if epoch % disp_step == 0 or epoch == 1:
            #Calculate batch loss and accuracy
            batch_loss, acc = sess.run([loss, accuracy], 
                                       feed_dict={X: batch_xs, y: batch_ys})
            print('Epoch: ', epoch, 'Minibatch Loss: ', batch_loss, 
                  'Training acc: ', acc)
    print('Optimization Finished.')
    
    #print('Test acc')

Epoch:  0 Minibatch Loss:  3541.9287 Training acc:  0.13333334
Epoch:  1 Minibatch Loss:  2959.593 Training acc:  0.2
Epoch:  100 Minibatch Loss:  232.5296 Training acc:  0.13333334
Epoch:  200 Minibatch Loss:  46.315056 Training acc:  0.4
Epoch:  300 Minibatch Loss:  47.62943 Training acc:  0.46666667
Epoch:  400 Minibatch Loss:  19.02382 Training acc:  0.53333336
Epoch:  500 Minibatch Loss:  20.137306 Training acc:  0.46666667
Epoch:  600 Minibatch Loss:  25.899374 Training acc:  0.46666667
Epoch:  700 Minibatch Loss:  33.81042 Training acc:  0.46666667
Epoch:  800 Minibatch Loss:  25.935343 Training acc:  0.46666667
Epoch:  900 Minibatch Loss:  11.607657 Training acc:  0.73333335
Epoch:  1000 Minibatch Loss:  36.567886 Training acc:  0.53333336
Epoch:  1100 Minibatch Loss:  19.32782 Training acc:  0.46666667
Epoch:  1200 Minibatch Loss:  11.63997 Training acc:  0.53333336
Epoch:  1300 Minibatch Loss:  15.066047 Training acc:  0.6
Epoch:  1400 Minibatch Loss:  20.351799 Training acc:

#### Run session

In [175]:
'''
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())


writer = tf.summary.FileWriter(logs_path, sess.graph)
merged_summaries = tf.summary.merge_all()
        
print('Commence Training...')

total_batch = int(x_train.shape[0] / batch_size)
    
#Run number of epochs
for epoch in range(epochs):
    avg_loss = 0
        
    #Loop over the batchs for each epoch
    for i in range(batch_count):
        batch_xs = x_train[(i*batch_size):(i+batch_size)+batch_size-1]
        batch_ys = y_train_hot[(i*batch_size):(i+batch_size)+batch_size-1]
            
        _, c = sess.run([optimizer, loss], feed_dict={X: batch_xs, y: batch_ys})
            
        if i % 25 == 0:
            s = sess.run(merged_summaries, feed_dict={X: batch_xs, y: batch_ys})
            writer.add_summary(s, epoch*total_batch + i)
                
        avg_loss += c / total_batch
            
    if(epoch+1) % disp_step == 0:
        print("Epoch: ", '%04d' % (epoch+1), "loss=", "{:.9f}".format(avg_loss))
            
print('Optimization finished.')
    
### EVALUTATE
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
print('Accuracy: ', sess.run(accuracy, feed_dict={X: x_train, y: y_train_hot}))
'''

'\nsaver = tf.train.Saver()\nsess = tf.Session()\nsess.run(tf.global_variables_initializer())\n\n\nwriter = tf.summary.FileWriter(logs_path, sess.graph)\nmerged_summaries = tf.summary.merge_all()\n        \nprint(\'Commence Training...\')\n\ntotal_batch = int(x_train.shape[0] / batch_size)\n    \n#Run number of epochs\nfor epoch in range(epochs):\n    avg_loss = 0\n        \n    #Loop over the batchs for each epoch\n    for i in range(batch_count):\n        batch_xs = x_train[(i*batch_size):(i+batch_size)+batch_size-1]\n        batch_ys = y_train_hot[(i*batch_size):(i+batch_size)+batch_size-1]\n            \n        _, c = sess.run([optimizer, loss], feed_dict={X: batch_xs, y: batch_ys})\n            \n        if i % 25 == 0:\n            s = sess.run(merged_summaries, feed_dict={X: batch_xs, y: batch_ys})\n            writer.add_summary(s, epoch*total_batch + i)\n                \n        avg_loss += c / total_batch\n            \n    if(epoch+1) % disp_step == 0:\n        print("Epoc